In [ ]:
import pandas as pd
import json
import numpy as np
import ast
import os

In [ ]:
datos_items = pd.read_csv('C:\\Users\\Alañ\\Documents\\Henry\\Proyecto 1\\Data\\usuarios_items_parte_2.csv')
datos_juegos = pd.read_csv('C:\\Users\\Alañ\\Documents\\Henry\\Proyecto 1\\Data\\Juedos_steam.csv')
datos_review = pd.read_csv('C:\\Users\\Alañ\\Documents\\Henry\\Proyecto 1\\Data\\recomendaciones_review_usuarios.csv')


In [ ]:
datos_items.columns

In [ ]:
datos_juegos.columns

In [ ]:
datos_review.columns

In [ ]:
print(datos_juegos[datos_juegos['id.1'] == 10])

In [ ]:
def PlayTimeGenre( genero: str):
    lista_años = pd.DataFrame()
    for j in range(0, len(datos_items)):
        if datos_items.loc[j, 'item_id'] in datos_juegos['id.1']:
            juego = datos_juegos.loc[datos_juegos['id.1']== datos_items.loc[j, 'item_id']]
            generos = str(juego['genres'])
            if genero in generos:
                if str(juego['release_date'])[9:14][0] == '2':
                    año = str(juego['release_date'])[9:13]
                elif str(juego['release_date'])[9:14][0] == ' ':
                    año = str(juego['release_date'])[10:14]
                cantidad = datos_items.loc[j, 'playtime_forever']
                if año in lista_años.columns:
                    lista_años[año] += [cantidad]
                else:
                    lista_años[año] = [cantidad]
    año = 0
    cantidad = 0            
    for i in lista_años.columns:
        if lista_años.loc[0, i] > cantidad:
            cantidad = lista_años.loc[0, i]
            año = i
    return print('El genero', genero, 'tiene más cantidad de hora en los juegos lanzados en el año:', año, 'con una cantidad de', cantidad, 'horas.')


In [ ]:
def UserForGenre( genero : str ):
    lista_usuarios_años  = pd.DataFrame()
    user_max = ' '
    user_temp = ' '
    desde_max = -1
    hasta_max = -1
    desde_temp = 0
    cantidad_horas_max = 0
    cantidad_horas_temp = 0

    for j in range(0, 1000):
        if user_temp != datos_items.loc[j, 'user_id']:
            if cantidad_horas_temp > cantidad_horas_max:
                cantidad_horas_max = cantidad_horas_temp
                user_max = user_temp
                hasta_max = j-1 
                desde_max = desde_temp

            cantidad_horas_temp = 0
            desde_temp = 0
            user_temp = datos_items.loc[j, 'user_id']

        if desde_temp == 0:
            desde_temp = j

        if datos_items.loc[j, 'item_id'] in datos_juegos['id.1']:
            juego = datos_juegos.loc[datos_juegos['id.1']== datos_items.loc[j, 'item_id']]
            generos = str(juego['genres'])
            if genero in generos:
                cantidad_horas_temp += datos_items.loc[j,'playtime_forever']

    lista_usuarios_años = pd.DataFrame()

    for j in range(desde_max, hasta_max):
        if datos_items.loc[j, 'item_id'] in datos_juegos['id.1']:
            juego = datos_juegos.loc[datos_juegos['id.1']== datos_items.loc[j, 'item_id']]
            generos = str(juego['genres'])
            if genero in generos:
                if str(juego['release_date'])[9:14][0] == '2':
                    año = str(juego['release_date'])[9:13]
                elif str(juego['release_date'])[9:14][0] == ' ':
                    año = str(juego['release_date'])[10:14]
                cantidad = datos_items.loc[j,'playtime_forever']
                if año in lista_usuarios_años.columns:
                    lista_usuarios_años[año] += [cantidad]
                else:
                    lista_usuarios_años[año] = [cantidad]
    lista_final_usuario = []
    for i in lista_usuarios_años.columns:
        año = i
        horas = lista_usuarios_años.loc[0, i]
        lista_final_usuario.append ({año: horas})
            
    return print('Usuario con más horas jugadas para el genero', genero, 'es', user_max,'y esta es la cantidad por años', lista_final_usuario)

In [226]:
def UsersRecommend( año : int ):
    lista_mejores_juegos ={'1ro': ['',0], '2do':['',0], '3ro':['',0]}
    todos_juegos = {}
    for j in range(len(datos_review)):
        juego = datos_juegos.loc[datos_juegos['id.1']== datos_review.loc[j, 'item_id']]
        if str(año) in (str(juego['release_date'])[9:14]):
            if datos_review.loc[j, 'item_id'] in todos_juegos:
                if datos_review.loc[j, 'review'] == '2':
                    todos_juegos[datos_review.loc[j, 'item_id']][1]  +=1
                todos_juegos[datos_review.loc[j, 'item_id']][2]  +=1
            else:
                todos_juegos[datos_review.loc[j, 'item_id']] = [datos_review.loc[j, 'item_id'],0,1]
    todos_juegos = pd.DataFrame(todos_juegos)
    for i in range(0, len(todos_juegos)):
        calculo = (todos_juegos.loc[1, todos_juegos.columns[i]]*100/todos_juegos.loc[2, todos_juegos.columns[i]])
        if calculo > lista_mejores_juegos['1ro'][1]:
            lista_mejores_juegos['3ro'] = lista_mejores_juegos['2do']
            lista_mejores_juegos['2do'] = lista_mejores_juegos['1ro']
            lista_mejores_juegos['1ro'] = [todos_juegos.loc[0, todos_juegos.columns[i]],calculo]
        elif calculo > lista_mejores_juegos['2do'][1]:
            lista_mejores_juegos['3ro'] = lista_mejores_juegos['2do']
            lista_mejores_juegos['2do'] = [todos_juegos.loc[0, todos_juegos.columns[i]],calculo]
        elif calculo > lista_mejores_juegos['3ro'][1]:
            lista_mejores_juegos['3ro'] = [todos_juegos.loc[0, todos_juegos.columns[i]],calculo]
    return lista_mejores_juegos

In [227]:
UsersRecommend(2000)

{'1ro': [10, 96.49122807017544],
 '2do': [3730, 83.33333333333333],
 '3ro': [6910, 82.3529411764706]}

In [245]:
def UsersWorstDeveloper( año : int ):
    lista_peores_developers ={'1ro': ['',0], '2do':['',0], '3ro':['',0]}
    todos_developers = {}
    for j in range(len(datos_review)):
        juego = datos_juegos.loc[datos_juegos['id.1']== datos_review.loc[j, 'item_id']]
        if str(año) in (str(juego['release_date'])[9:14]):
            if str(juego['developer'])[10:-31] in todos_developers:
                if datos_review.loc[j, 'review'] == '0':
                    todos_developers[str(juego['developer'])[10:-31]][0] +=1
                todos_developers[str(juego['developer'])[10:-31]][1] +=1
            else:
                todos_developers[str(juego['developer'])[10:-31]] = [1, 1]
    todos_developers = pd.DataFrame(todos_developers)
    for i in range(0, len(todos_developers)):
        calculo = (todos_developers.loc[0, todos_developers.columns[i]]*100/todos_developers.loc[1, todos_developers.columns[i]])
        if calculo > lista_peores_developers['1ro'][1]:
            lista_peores_developers['3ro'] = lista_peores_developers['2do']
            lista_peores_developers['2do'] = lista_peores_developers['1ro']
            lista_peores_developers['1ro'] = [todos_developers.columns[i],calculo]
        elif calculo > lista_peores_developers['2do'][1]:
            lista_peores_developers['3ro'] = lista_peores_developers['2do']
            lista_peores_developers['2do'] = [todos_developers.columns[i],calculo]
        elif calculo > lista_peores_developers['3ro'][1]:
            lista_peores_developers['3ro'] = [todos_developers.columns[i],calculo]
    return lista_peores_developers

In [246]:
UsersWorstDeveloper(2010)

{'1ro': ['ohemia Interactive', 1.1235955056179776],
 '2do': ['TaleWorlds Entertainment', 0.4716981132075472],
 '3ro': ['', 0]}

In [235]:
def sentiment_analysis( empresa_desarrolladora : str ):
    Negativo = 0
    Neutro = 0
    Positivo = 0
    for j in range(len(datos_review)):
        juego = datos_juegos.loc[datos_juegos['id.1']== datos_review.loc[j, 'item_id']]
        if str(juego['publisher'])[9:-31] == empresa_desarrolladora:
            if datos_review.loc[j, 'review'] == '0':
                Negativo += 1
            elif datos_review.loc[j, 'review'] == '1':
                Neutro += 1
            elif datos_review.loc[j, 'review'] == '2':
                Positivo += 1
    return {empresa_desarrolladora: str('Negativo=' + str(Negativo) +', Neutro=' + str(Neutro) + ', Positivo=' + str(Positivo))}

In [236]:
sentiment_analysis('Valve')

{'Valve': 'Negativo=0, Neutro=6, Positivo=7004'}